In [2]:
import pandas as pd
from urllib import request, parse
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
import warnings
warnings.simplefilter('ignore')
import urllib.request
import json  
import dill
import numpy as np

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [3]:
X_test.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,8879-ZKJOF,Female,0,No,No,41,Yes,No,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Bank transfer (automatic),79.85,3320.75
1,0201-MIBOL,Female,1,No,No,66,Yes,Yes,Fiber optic,Yes,No,No,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),102.40,6471.85
2,1600-DILPE,Female,0,No,No,12,Yes,No,DSL,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),45.00,524.35


In [4]:
def get_prediction(x):
    customerID, gender, SeniorCitizen, Partner, Dependents, tenure, PhoneService, MultipleLines, InternetService, \
    OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies, Contract, PaperlessBilling,\
    PaymentMethod, MonthlyCharges, TotalCharges = x
    body = {'customerID': customerID, 
            'gender': gender,
            'SeniorCitizen': SeniorCitizen, 
            'Partner': Partner, 
            'Dependents': Dependents,
            'tenure': tenure, 
            'PhoneService': PhoneService, 
            'MultipleLines': MultipleLines, 
            'InternetService': InternetService,
            'OnlineSecurity': OnlineSecurity, 
            'OnlineBackup': OnlineBackup, 
            'DeviceProtection': DeviceProtection, 
            'TechSupport': TechSupport,
            'StreamingTV': StreamingTV, 
            'StreamingMovies': StreamingMovies, 
            'Contract': Contract, 
            'PaperlessBilling': PaperlessBilling,
            'PaymentMethod': PaymentMethod, 
            'MonthlyCharges': MonthlyCharges, 
            'TotalCharges': TotalCharges
            } 

    myurl = "http://127.0.0.1:5000/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    r = json.loads(response.read())
    return r['forecast'], round(r['predictions'], 3)

#### Единичный запрос:

поскольку на Step 1 был подсчитан оптимальный порог отнесения клиента к оттоку thr = 0.366063, этот порог использован для выдачи прогноза по оттоку. probability score тоже оставляю для наглядности

In [5]:
get_prediction(('', 'дшщ з', 0, 'No', 'No', '', 'Yes', 'No', 'DSL', 'Yes', 'No', 'Yes', '', '', '', '', 'Yes', '', '', '3320.7'))

('с клиентом всё хорошо.', 0.073)

#### Несколько запросов из X_test

In [6]:
X_test.iloc[:30].apply(lambda x: get_prediction(x), 1)

0         (с клиентом всё хорошо., 0.099)
1         (с клиентом всё хорошо., 0.139)
2         (с клиентом всё хорошо., 0.298)
3     (клиент может уйти в отток!, 0.434)
4         (с клиентом всё хорошо., 0.187)
5         (с клиентом всё хорошо., 0.176)
6         (с клиентом всё хорошо., 0.332)
7     (клиент может уйти в отток!, 0.636)
8         (с клиентом всё хорошо., 0.125)
9         (с клиентом всё хорошо., 0.235)
10    (клиент может уйти в отток!, 0.419)
11        (с клиентом всё хорошо., 0.225)
12    (клиент может уйти в отток!, 0.649)
13    (клиент может уйти в отток!, 0.392)
14        (с клиентом всё хорошо., 0.029)
15        (с клиентом всё хорошо., 0.017)
16        (с клиентом всё хорошо., 0.364)
17    (клиент может уйти в отток!, 0.434)
18    (клиент может уйти в отток!, 0.454)
19        (с клиентом всё хорошо., 0.328)
20         (с клиентом всё хорошо., 0.32)
21     (клиент может уйти в отток!, 0.41)
22        (с клиентом всё хорошо., 0.138)
23        (с клиентом всё хорошо.,